# Calibration of climate parameters

In [1]:
import yaml
import numpy as np
import pandas as pd
from scipy.stats import linregress
from mce.core.climate_parms import (
    ParmEstimate, get_rwf_ramp, get_amp_full, get_ebm, ebm2irm,
)

## Normalized CMIP data

In [2]:
with pd.HDFStore('data/cmip_normalized.h5', 'r') as store:
    df_norm_cmip5 = store['CMIP5/climate_norm1']
    df_norm_cmip6 = store['CMIP6/climate_norm1']

## Calibration of impulse response parameters

In [3]:
# nl = 2 # 2-layer model
nl = 3 # 3-layer model

obj = ParmEstimate(nl=nl)
time = np.arange(150) + 0.5
df_parms = {}

# Loop over CMIP models
for mip, df_norm in zip(
    ['CMIP5', 'CMIP6'],
    [
        df_norm_cmip5.rename({'abrupt4xCO2': 'abrupt-4xCO2'}, level=1),
        df_norm_cmip6,
    ],
):
    for source_id, df in df_norm.groupby(level=0):
        if df.shape[0] != 6:
            continue

        df = df.droplevel(0)
        data = [
            df.loc[('abrupt-4xCO2', 'rtnt')].dropna().values,
            df.loc[('abrupt-4xCO2', 'tas')].dropna().values,
            df.loc[('1pctCO2', 'rtnt')].dropna().values,
            df.loc[('1pctCO2', 'tas')].dropna().values,
        ]
        if [len(d1) for d1 in data] != [150, 150, 140, 140]:
            raise ValueError('Invalid data length')

        data_std = [
            df.loc[('piControl', 'rtnt')].dropna().std(ddof=0),
            df.loc[('piControl', 'tas')].dropna().std(ddof=0),
        ]

        # MCE calibration
        ret = obj.minimize_wrap(time, data, data_std)
        # Conventional linear regression for comparison
        regress = linregress(data[1], data[0])

        df_parms[(mip, source_id)] = pd.Series({
            'co2_alpha': ret[0],
            'co2_beta': ret[1],
            'lambda': ret[2],
            'lambda_reg': -regress.slope,
            'q4x_reg': regress.intercept,
            'tcr_gcm': data[3][70-10:70+10].mean(),
            **{f'a{j}': x for j, x in enumerate(ret[3])},
            **{f'tau{j}': x for j, x in enumerate(ret[4])},
        })

df_parms = pd.concat(df_parms).unstack()

In [4]:
df_parms

co2_alpha  co2_beta    lambda  lambda_reg   q4x_reg  \
CMIP5 ACCESS1.0     4.246654  1.019410  0.784776    0.777060  5.949061   
      ACCESS1.3     3.925020  1.075596  0.834143    0.819500  5.788829   
      BCC-CSM1.1    4.450616  1.037867  1.130385    1.137433  6.429641   
      BNU-ESM       5.453729  1.007902  0.970155    0.966931  7.588264   
      CCSM4         4.917046  1.040323  1.189000    1.227101  7.256500   
...                      ...       ...       ...         ...       ...   
CMIP6 NorESM2-LM    4.070018  1.178750  1.285577    1.380486  7.003989   
      NorESM2-MM    4.296143  1.211978  1.428425    1.472066  7.356114   
      SAM0-UNICON   5.252838  1.006970  0.958214    1.048148  7.791731   
      TaiESM1       5.293819  1.002893  0.824424    0.863822  7.587014   
      UKESM1-0-LL   5.222636  1.006065  0.681346    0.679551  7.253754   

                    tcr_gcm        a0        a1        a2      tau0  \
CMIP5 ACCESS1.0    1.921747  0.309701  0.186391  0.503909  1.771025   
      ACCESS1.3    1.631630  0.183191  0.311308  0.505501  1.084692   
      BCC-CSM1.1   1.736432  0.349640  0.278756  0.371604  1.677255   
      BNU-ESM      2.490674  0.288967  0.349246  0.361787  1.255106   
      CCSM4        1.769586  0.167795  0.406360  0.425844  0.207641   
...                     ...       ...       ...       ...       ...   
CMIP6 NorESM2-LM   1.475827  0.071654  0.519140  0.409207  0.009144   
      NorESM2-MM   1.335450  0.146186  0.483434  0.370380  0.065719   
      SAM0-UNICON  2.264424  0.079876  0.471748  0.448376  0.349683   
      TaiESM1      2.357541  0.194131  0.350745  0.455124  0.829860   
      UKESM1-0-LL  2.786460  0.264903  0.308512  0.426585  1.466066   

                        tau1        tau2  
CMIP5 ACCESS1.0    11.862830  219.728921  
      ACCESS1.3     7.637977  255.383861  
      BCC-CSM1.1    9.770734  141.682897  
      BNU-ESM       9.066619  291.727750  
      CCSM4         4.378526  170.572083  
...                      ...         ...  
CMIP6 NorESM2-LM    1.822130  228.450360  
      NorESM2-MM    2.321406  217.502799  
      SAM0-UNICON   3.997234  265.292199  
      TaiESM1       8.347437  262.912735  
      UKESM1-0-LL  18.557701  317.950803  

[68 rows x 12 columns]

In [5]:
with open('mce/core/attributes.yml', 'r') as f1:
    atts = yaml.safe_load(f1)

In [6]:
def mkdesc(df, atts):
    for name in df:
        if name in atts['forcing']:
            att1 = atts['forcing'][name]
        else:
            att1 = atts['climate'][name]
        print('{}: {} ({})'.format(name, att1['long_name'], att1['units']))

In [7]:
mkdesc(df_parms, atts['parameters'])

co2_alpha: Scaling factor of CO2 forcing (W m-2)
co2_beta: Amplification factor of CO2 forcing (no_unit)
lambda: Climate feedback parameter (W m-2 K-1)
lambda_reg: Climate feedback parameter based on conventional linear regression (W m-2 K-1)
q4x_reg: 4xCO2 forcing based on conventional linear regression (W m-2)
tcr_gcm: Transient climate response by GCM data (K)
a0: Fraction of the first time-constant contribution to the surface temperature change (no_unit)
a1: Fraction of the second time-constant contribution to the surface temperature change (no_unit)
a2: Fraction of the third time-constant contribution to the surface temperature change (no_unit)
tau0: First element of time constants (year)
tau1: Second element of time constants (year)
tau2: Third element of time constants (year)


## Derived parameters

In [8]:
# ECS and TCR derived analytically
ecs = df_parms['co2_alpha'] * np.log(2) / df_parms['lambda']
tp70 = np.log(2) / np.log(1.01)
# Realized warming fraction at 2xCO2 in a 1%-per-year increase trajectory
rwf = get_rwf_ramp(df_parms, tp70)
df_ecs_tcr = pd.DataFrame({
    'ecs': ecs,
    'tcr': ecs * rwf.squeeze(),
})
df_ecs_tcr

ecs       tcr
CMIP5 ACCESS1.0    3.750823  1.982799
      ACCESS1.3    3.261569  1.697960
      BCC-CSM1.1   2.729099  1.798378
      BNU-ESM      3.896530  2.445166
      CCSM4        2.866472  1.789678
...                     ...       ...
CMIP6 NorESM2-LM   2.194439  1.390633
      NorESM2-MM   2.084715  1.390147
      SAM0-UNICON  3.799767  2.196986
      TaiESM1      4.450862  2.474002
      UKESM1-0-LL  5.313093  2.821650

[68 rows x 2 columns]

In [9]:
mkdesc(df_ecs_tcr, atts['parameters'])

ecs: Equilibrium climate sensitivity (K)
tcr: Transient climate response (K)


In [10]:
# Derived energy balance model parameters
df_ebm = get_ebm(df_parms)
df_ebm

gamma1    gamma2       xis        xi1         xi2
CMIP5 ACCESS1.0     1.326344  1.143810  4.068806  21.163595   63.826310
      ACCESS1.3     2.165397  1.128747  3.941619  11.607373   94.285380
      BCC-CSM1.1    1.374457  0.815835  4.717565  14.010282   44.528527
      BNU-ESM       1.527755  0.724717  3.593259  10.862076   91.361924
      CCSM4         4.511710  0.976175  1.316126   7.767238   79.439224
...                      ...       ...       ...        ...         ...
CMIP6 NorESM2-LM   15.414933  0.918182  0.158261   3.723841  117.515024
      NorESM2-MM    6.751687  0.922244  0.586778   4.507207  111.594692
      SAM0-UNICON   4.439912  0.879192  2.752442   4.648134  108.413236
      TaiESM1       2.243450  0.828708  2.968949  10.748200   87.478163
      UKESM1-0-LL   1.462061  0.559844  3.429777  18.959723   74.189005

[68 rows x 5 columns]

In [11]:
mkdesc(df_ebm, atts['parameters'])

gamma1: Coefficient of heat exchange between the surface and first sub-surface layers (W m-2 K-1)
gamma2: Coefficient of heat exchange between the first and second sub-surface layers (W m-2 K-1)
xis: Heat capacity of the surface layer divided by annual total seconds (J m-2 K-1 s-1)
xi1: Heat capacity of the first sub-surface layer divided by annual total seconds (J m-2 K-1 s-1)
xi2: Heat capacity of the second sub-surface layer divided by annual total seconds (J m-2 K-1 s-1)


In [12]:
# Derive sub-surface amplitutde parameters
df_amp_full = get_amp_full(df_parms)
df_amp_full

a10       a11       a12           a20       a21  \
CMIP5 ACCESS1.0   -0.043503  0.248475  0.795028  1.425962e-03 -0.067085   
      ACCESS1.3   -0.053663  0.357038  0.696624  7.060066e-04 -0.035933   
      BCC-CSM1.1  -0.078307  0.410089  0.668218  2.482660e-03 -0.089420   
      BNU-ESM     -0.069038  0.480426  0.588613  6.942570e-04 -0.037230   
      CCSM4       -0.023719  0.486378  0.537342  6.067609e-05 -0.027658   
...                     ...       ...       ...           ...       ...   
CMIP6 NorESM2-LM  -0.002825  0.559510  0.443316  2.018675e-07 -0.008081   
      NorESM2-MM  -0.016205  0.567614  0.448591  8.805909e-06 -0.011102   
      SAM0-UNICON -0.044492  0.500397  0.544096  1.265307e-04 -0.016764   
      TaiESM1     -0.044112  0.424030  0.620082  3.495372e-04 -0.036411   
      UKESM1-0-LL -0.035519  0.413285  0.622234  3.973476e-04 -0.067301   

                        a22  
CMIP5 ACCESS1.0    1.065659  
      ACCESS1.3    1.035227  
      BCC-CSM1.1   1.086937  
      BNU-ESM      1.036535  
      CCSM4        1.027597  
...                     ...  
CMIP6 NorESM2-LM   1.008081  
      NorESM2-MM   1.011094  
      SAM0-UNICON  1.016638  
      TaiESM1      1.036061  
      UKESM1-0-LL  1.066904  

[68 rows x 6 columns]

In [13]:
mkdesc(df_amp_full, atts['parameters'])

a10: Fraction of the first time-constant contribution to the first sub-surface temperature change (no_unit)
a11: Fraction of the second time-constant contribution to the first sub-surface temperature change (no_unit)
a12: Fraction of the third time-constant contribution to the first sub-surface temperature change (no_unit)
a20: Fraction of the first time-constant contribution to the second sub-surface temperature change (no_unit)
a21: Fraction of the second time-constant contribution to the second sub-surface temperature change (no_unit)
a22: Fraction of the third time-constant contribution to the second sub-surface temperature change (no_unit)


## Validation

In [14]:
# Convert the energy balance model parameters to impulse response parameters
df_irm = ebm2irm(pd.concat([df_parms[['lambda']], df_ebm], axis=1))
df_irm

tau0       tau1        tau2        a0        a1  \
CMIP5 ACCESS1.0    1.771025  11.862830  219.728921  0.309701  0.186391   
      ACCESS1.3    1.084692   7.637977  255.383861  0.183191  0.311308   
      BCC-CSM1.1   1.677255   9.770734  141.682897  0.349640  0.278756   
      BNU-ESM      1.255106   9.066619  291.727750  0.288967  0.349246   
      CCSM4        0.207641   4.378526  170.572083  0.167795  0.406360   
...                     ...        ...         ...       ...       ...   
CMIP6 NorESM2-LM   0.009144   1.822130  228.450360  0.071654  0.519140   
      NorESM2-MM   0.065719   2.321406  217.502799  0.146186  0.483434   
      SAM0-UNICON  0.349683   3.997234  265.292199  0.079876  0.471748   
      TaiESM1      0.829860   8.347437  262.912735  0.194131  0.350745   
      UKESM1-0-LL  1.466066  18.557701  317.950803  0.264903  0.308512   

                         a2       a10       a11       a12           a20  \
CMIP5 ACCESS1.0    0.503909 -0.043503  0.248475  0.795028  1.425962e-03   
      ACCESS1.3    0.505501 -0.053663  0.357038  0.696624  7.060066e-04   
      BCC-CSM1.1   0.371604 -0.078307  0.410089  0.668218  2.482660e-03   
      BNU-ESM      0.361787 -0.069038  0.480426  0.588613  6.942570e-04   
      CCSM4        0.425844 -0.023719  0.486378  0.537342  6.067609e-05   
...                     ...       ...       ...       ...           ...   
CMIP6 NorESM2-LM   0.409207 -0.002825  0.559510  0.443316  2.018675e-07   
      NorESM2-MM   0.370380 -0.016205  0.567614  0.448591  8.805909e-06   
      SAM0-UNICON  0.448376 -0.044492  0.500397  0.544096  1.265307e-04   
      TaiESM1      0.455124 -0.044112  0.424030  0.620082  3.495372e-04   
      UKESM1-0-LL  0.426585 -0.035519  0.413285  0.622234  3.973476e-04   

                        a21       a22  
CMIP5 ACCESS1.0   -0.067085  1.065659  
      ACCESS1.3   -0.035933  1.035227  
      BCC-CSM1.1  -0.089420  1.086937  
      BNU-ESM     -0.037230  1.036535  
      CCSM4       -0.027658  1.027597  
...                     ...       ...  
CMIP6 NorESM2-LM  -0.008081  1.008081  
      NorESM2-MM  -0.011102  1.011094  
      SAM0-UNICON -0.016764  1.016638  
      TaiESM1     -0.036411  1.036061  
      UKESM1-0-LL -0.067301  1.066904  

[68 rows x 12 columns]

In [15]:
if nl == 2:
    names1 = ['tau0', 'tau1', 'a0', 'a1']
    names2 = ['a10', 'a11']
else:
    names1 = ['tau0', 'tau1', 'tau2', 'a0', 'a1', 'a2']
    names2 = ['a10', 'a11', 'a12', 'a20', 'a21', 'a22']

(
    np.allclose(df_parms[names1], df_irm[names1]),
    np.allclose(df_amp_full[names2], df_irm[names2]),
)

(True, True)

## Save the results

In [16]:
outpath = 'data/parms_calib_climate.h5'
if nl == 2:
    key = 'cmip_norm1__nl2'
else:
    key = 'cmip_norm1__nl3'

pd.concat([
    df_parms, df_ecs_tcr, df_amp_full, df_ebm,
], axis=1).sort_index(axis=1).to_hdf(outpath, key=key)

## Comparison with the previous results

The results depend on slight numerical differences of input data and Python package versions.

In [17]:
from netCDF4 import Dataset

In [18]:
df_chk = {}

In [19]:
with Dataset('mce/data/parms/parms_irm-3_rtnt-tas_cmip5.nc') as ncf:
    df_chk['CMIP5'] = pd.DataFrame({
        k: v[:].filled()
        for k, v in ncf.variables.items() if k not in ['dataset']
    }, index=[
        ''.join(x.astype(str)).strip()
        for x in ncf.variables['dataset'][:]
    ])

In [20]:
with Dataset('mce/data/parms/parms_irm-3_rtnt-tas_cmip6.nc') as ncf:
    df_chk['CMIP6'] = pd.DataFrame({
        k: v[:].filled()
        for k, v in ncf.variables.items() if k not in ['dataset']
    }, index=[
        ''.join(x.astype(str)).strip()
        for x in ncf.variables['dataset'][:]
    ])

In [22]:
map_name = {
    'time_constant_0': 'tau0',
    'time_constant_1': 'tau1',
    'time_constant_2': 'tau2',
    'amplitude_0': 'a0',
    'amplitude_1': 'a1',
    'amplitude_2': 'a2',
    'alpha': 'co2_alpha',
    'beta': 'co2_beta',
}
df_chk = pd.concat(df_chk).rename(columns=map_name)
df_chk

tau0        a0       tau1  lambda_reg   tcr_gcm  \
CMIP5 ACCESS1.0    1.770429  0.309628  11.857424    0.777060  1.921747   
      ACCESS1.3    1.085171  0.183242   7.639266    0.819500  1.631630   
      BCC-CSM1.1   1.676995  0.349606   9.769733    1.137433  1.736432   
      BNU-ESM      1.253887  0.288810   9.062490    0.966931  2.490674   
      CanESM2      2.171483  0.447780  21.435611    1.038722  2.340212   
...                     ...       ...        ...         ...       ...   
CMIP6 NESM3        0.688722  0.383838  18.372645    0.801938  2.709527   
      NorESM2-LM   0.043158  0.071591   1.821879    1.380486  1.475827   
      NorESM2-MM   0.066863  0.146193   2.321184    1.472066  1.335450   
      SAM0-UNICON  0.349018  0.079816   3.996651    1.048148  2.264424   
      UKESM1-0-LL  1.464433  0.264797  18.546190    0.679551  2.786460   

                    ecs_reg        a2        a1  co2_beta    lambda  \
CMIP5 ACCESS1.0    3.827931  0.503922  0.186450  1.019410  0.784780   
      ACCESS1.3    3.531926  0.505497  0.311261  1.075596  0.834141   
      BCC-CSM1.1   2.826381  0.371607  0.278788  1.037867  1.130386   
      BNU-ESM      3.923890  0.361799  0.349391  1.007903  0.970174   
      CanESM2      3.691430  0.329360  0.222860  1.018086  1.018417   
...                     ...       ...       ...       ...       ...   
CMIP6 NESM3        4.715919  0.346096  0.270066  1.048372  0.870825   
      NorESM2-LM   2.536783  0.409216  0.519193  1.178675  1.285466   
      NorESM2-MM   2.498568  0.370381  0.483427  1.211972  1.428417   
      SAM0-UNICON  3.716904  0.448376  0.471808  1.006970  0.958230   
      UKESM1-0-LL  5.337168  0.426639  0.308563  1.006066  0.681362   

                         tau2       ecs  co2_alpha       tcr  
CMIP5 ACCESS1.0    219.715468  3.750815   4.246666  1.982800  
      ACCESS1.3    255.390396  3.261574   3.925013  1.697960  
      BCC-CSM1.1   141.681350  2.729098   4.450617  1.798378  
      BNU-ESM      291.686972  3.896504   5.453803  2.445168  
      CanESM2      261.563893  3.642714   5.352112  2.298461  
...                       ...       ...        ...       ...  
CMIP6 NESM3        411.292266  4.434479   5.571190  2.697093  
      NorESM2-LM   228.466186  2.194624   4.070008  1.390649  
      NorESM2-MM   217.483061  2.084728   4.296145  1.390161  
      SAM0-UNICON  265.279760  3.799744   5.252893  2.196989  
      UKESM1-0-LL  317.829576  5.313040   5.222703  2.821658  

[63 rows x 14 columns]

In [23]:
with pd.HDFStore(outpath, 'r') as store:
    df_new = store['cmip_norm1__nl3']

In [44]:
names = ['tau0', 'tau1', 'tau2', 'a0', 'a1', 'a2', 'lambda']
models_chk = []
for k, d1 in df_chk.iterrows():
    if not np.allclose(
        d1.loc[names], df_new.loc[k, names],
        atol=1e-3,
        rtol=1e-3,
    ):
        models_chk.append(k)

models_chk

[('CMIP5', 'FGOALS-s2'),
 ('CMIP6', 'ACCESS-CM2'),
 ('CMIP6', 'CESM2-WACCM-FV2'),
 ('CMIP6', 'GFDL-CM4'),
 ('CMIP6', 'NorESM2-LM'),
 ('CMIP6', 'NorESM2-MM')]

In [45]:
df_chk.loc[models_chk, names]

tau0       tau1         tau2        a0        a1  \
CMIP5 FGOALS-s2        0.003988   4.661324   315.523031  0.091069  0.441018   
CMIP6 ACCESS-CM2       2.164487  37.497733  3643.927675  0.322753  0.383433   
      CESM2-WACCM-FV2  0.025156   5.327224   310.716823  0.082376  0.327127   
      GFDL-CM4         1.005830   6.924113   213.156880  0.276566  0.242186   
      NorESM2-LM       0.043158   1.821879   228.466186  0.071591  0.519193   
      NorESM2-MM       0.066863   2.321184   217.483061  0.146193  0.483427   

                             a2    lambda  
CMIP5 FGOALS-s2        0.467913  0.884644  
CMIP6 ACCESS-CM2       0.293814  0.721581  
      CESM2-WACCM-FV2  0.590498  0.586777  
      GFDL-CM4         0.481249  0.808709  
      NorESM2-LM       0.409216  1.285466  
      NorESM2-MM       0.370381  1.428417

In [46]:
df_new.loc[models_chk, names]

tau0       tau1         tau2        a0        a1  \
CMIP5 FGOALS-s2        0.000763   4.661471   315.520901  0.091078  0.441011   
CMIP6 ACCESS-CM2       2.164679  37.503335  3655.410523  0.322765  0.383461   
      CESM2-WACCM-FV2  0.006686   5.327341   310.706836  0.082379  0.327128   
      GFDL-CM4         0.921546   6.692471   217.812462  0.281724  0.251541   
      NorESM2-LM       0.009144   1.822130   228.450360  0.071654  0.519140   
      NorESM2-MM       0.065719   2.321406   217.502799  0.146186  0.483434   

                             a2    lambda  
CMIP5 FGOALS-s2        0.467910  0.884655  
CMIP6 ACCESS-CM2       0.293774  0.721581  
      CESM2-WACCM-FV2  0.590493  0.586801  
      GFDL-CM4         0.466735  0.834088  
      NorESM2-LM       0.409207  1.285577  
      NorESM2-MM       0.370380  1.428425